In [46]:
import os, re
import pandas as pd
import numpy as np

# #显示所有列
# pd.set_option('display.max_columns', None)

# #显示所有行
# pd.set_option('display.max_rows', None)

pd.set_option('max_colwidth',100) 

In [47]:
folders = [
    # r'D:\code\forecast_model\notebook\20250811_base',
    # r'D:\code\forecast_model\notebook\20250811_small_large',
    # r'D:\code\forecast_model\notebook\20250811_extraLarge',
    # r"D:\code\forecast_model\notebook\20250815_base_large",
    # r'D:\code\forecast_model\notebook\20250817_base',
    # r'D:\code\forecast_model\notebook\20250820_base',
    # r'D:\code\forecast_model\notebook\20250820_bigger',
    # r'D:\code\forecast_model\notebook\20250821_mlp_base2',
    # r'D:\code\forecast_model\notebook\20250822_tcn_baseline2',
    # r'D:\code\forecast_model\notebook\20250822_tcn_baseline3',
    # r"D:\code\forecast_model\notebook\20250824_tcn_baseline",
    # r"D:\code\forecast_model\notebook\20250824_tcn_baseline2",
    # r"D:\code\forecast_model\notebook\20250824_tcn_baseline3",
    # r'D:\code\forecast_model\notebook\20250825_tcn_baseline',
    # r"D:\code\forecast_model\notebook\20250826_tcn_filt_days",
    # r'D:\code\forecast_model\notebook\20250828_tcn_data',
    # r'D:\code\forecast_model\notebook\20250829_tcn_data',
    # r"D:\code\forecast_model\notebook\20250829_tcn_data2",
    # r'D:\code\forecast_model\notebook\20250830_tcn_data',
    # r'D:\code\forecast_model\notebook\20250831_tcn_only30',
    # r'D:\code\forecast_model\notebook\20250831_nofilter_top5',
    # r'D:\code\forecast_model\notebook\20250831_tcn_we',
    r'D:\code\forecast_model\notebook\20250831_tcn_we2',
    r'D:\code\forecast_model\notebook\20250901_warinup',
    ]

In [48]:
data = pd.DataFrame()
for folder in folders:
    for i in os.listdir(folder):
        if 'muilt_model' == i:continue
        # 20250811_base_P100_DeepLOB_v2_input_indepent	0.558035	2.77h
        # 20250811_base_P100_DeepLOB_v2_overfit
        if 'input_indepent' in i or 'overfit' in i: continue

        if i.endswith('.csv'): continue
        if i.endswith('.xlsx'): continue

        code = i.split('_')[3]
        fold = i.split('_')[4]

        train_folder = os.path.join(folder, i)
        if not os.path.isdir(train_folder): continue

        result_file = os.path.join(train_folder, 'result.csv')
        if not os.path.exists(result_file): continue

        # 修复替换
        _data = open(result_file, 'r').readlines()
        _data[1] = _data[1].replace('(30, 2, 1)', '30@2@1')
        with open(result_file, 'w') as f:
            f.writelines(_data)

        _data = pd.read_csv(result_file)
        # _data['describe.1'] = _data['describe.1'] + "_" + code + f"_fold{fold}"

        # 判断是否有nan
        if _data.isnull().values.any(): 
            print(i)
            continue

        if 'test_class_f1_0' in list(_data):
            _data[f'test_mean_class_f1'] = (_data[f'test_class_f1_0'] + _data[f'test_class_f1_1']) / 2
        elif 'test_final_class_f1_0' in list(_data):
            # 同时采用 best/final
            d2 = _data.copy()
            d2[f'test_mean_class_f1'] = _data[f'test_best_class_f1_0']
            for i in range(1, _data['y_n'][0] - 1):
                d2[f'test_mean_class_f1'] += _data[f'test_best_class_f1_{i}']
            d2[f'test_mean_class_f1'] = d2[f'test_mean_class_f1'] / (_data['y_n'][0] - 1)
            d2['train_title'] += '_best'

            _data[f'test_mean_class_f1'] = _data[f'test_final_class_f1_0']
            for i in range(1, _data['y_n'][0] - 1):
                _data[f'test_mean_class_f1'] += _data[f'test_final_class_f1_{i}']
            _data[f'test_mean_class_f1'] = _data[f'test_mean_class_f1'] / (_data['y_n'][0] - 1)
            _data['train_title'] += '_final'

            _data = pd.concat([_data, d2], ignore_index=True)

        # for _type in ['train', 'val', 'test_best', 'test_final', 'test_dummy']:
        #     _data[f'{_type}_mean_class_f1'] = (_data[f'{_type}_class_f1_0'] + _data[f'{_type}_class_f1_1']) / 2

        # for _type in ['train', 'val', 'test_best', 'test_final']:
            # _data[f'{_type}_mean_class_f1_enhanced_pct'] = 100 * (_data[f'{_type}_mean_class_f1'] - _data['test_dummy_mean_class_f1']) / _data['test_dummy_mean_class_f1']

        data = pd.concat([data, _data], ignore_index=True)

try:
    data['mean_test_01_f1'] = (data['test_best_class_f1_0'] + data['test_best_class_f1_1']) / 2
    data['mean_val_01_f1'] = (data['val_class_f1_0'] + data['val_class_f1_1']) / 2
except:
    pass

# 只使用 top 数据集 predict_n100
# data = data.loc[data['train_title'].str.contains('top5')].reset_index(drop=True)
# data = data.loc[data['train_title'].str.contains('predict_n100')].reset_index(drop=True)

data

,train_title,root,alist_upload_folder,amp,epochs,batch_n,batch_size,learning_rate,abs_learning_rate,no_better_stop,...,val_class_mcc_1_best,val_class_mcc_1,val_class_f1_1_best,val_class_f1_1,val_class_recall_1_best,val_class_recall_1,train_mean_class_f1,val_mean_class_f1,each_epoch_cost,cost
0,20250831_2_P100_seed0_wd0.0001_IDX2,20250831_2_P100_seed0_wd0.0001_IDX2,train_data,no,120,128,8192,0.0384,0,0,...,0.495882,0.431414,0.742789,0.716321,0.751649,0.708583,0.991606,0.715621,0.02h,1.65h
1,20250831_2_P100_seed0_wd0.001_IDX1,20250831_2_P100_seed0_wd0.001_IDX1,train_data,no,120,128,8192,0.0384,0,0,...,0.429371,0.424984,0.691016,0.714590,0.741811,0.703118,0.991508,0.712299,0.02h,1.65h
2,20250831_2_P100_seed0_wd0.01_IDX0,20250831_2_P100_seed0_wd0.01_IDX0,train_data,no,120,128,8192,0.0384,0,0,...,0.563581,0.421383,0.766069,0.713179,0.811789,0.700780,0.991370,0.710465,0.02h,1.64h
3,20250831_2_P100_seed1_wd0.0001_IDX5,20250831_2_P100_seed1_wd0.0001_IDX5,train_data,no,120,128,8192,0.0384,0,0,...,0.469839,0.420709,0.727987,0.712359,0.740801,0.701201,0.991746,0.710172,0.02h,1.22h
4,20250831_2_P100_seed1_wd0.001_IDX4,20250831_2_P100_seed1_wd0.001_IDX4,train_data,no,120,128,8192,0.0384,0,0,...,0.417502,0.442458,0.721820,0.724330,0.681500,0.709811,0.991796,0.720920,0.02h,1.22h
5,20250831_2_P100_seed1_wd0.01_IDX3,20250831_2_P100_seed1_wd0.01_IDX3,train_data,no,120,128,8192,0.0384,0,0,...,0.388090,0.431904,0.661841,0.718166,0.726034,0.706260,0.992289,0.715745,0.02h,1.64h
6,20250831_2_P100_seed2_wd0.0001_IDX8,20250831_2_P100_seed2_wd0.0001_IDX8,train_data,no,120,128,8192,0.0384,0,0,...,0.476126,0.432432,0.736792,0.719675,0.734084,0.704476,0.993034,0.715874,0.02h,1.21h
7,20250831_2_P100_seed2_wd0.001_IDX7,20250831_2_P100_seed2_wd0.001_IDX7,train_data,no,120,128,8192,0.0384,0,0,...,0.422287,0.424572,0.723761,0.714612,0.684205,0.702561,0.992014,0.712073,0.02h,1.23h
8,20250831_2_P100_seed2_wd0.01_IDX6,20250831_2_P100_seed2_wd0.01_IDX6,train_data,no,120,128,8192,0.0384,0,0,...,0.482327,0.430226,0.745306,0.717380,0.726426,0.705368,0.991794,0.714902,0.02h,1.23h
9,20250831_2_P100_seed3_wd0.0001_IDX11,20250831_2_P100_seed3_wd0.0001_IDX11,train_data,no,120,128,8192,0.0384,0,0,...,0.521579,0.429936,0.748408,0.717997,0.779789,0.703998,0.992751,0.714678,0.02h,1.20h


In [49]:
if len(folders) == 1:
    data.to_csv(os.path.join(folders[0], 'result.csv'), index=False)


In [50]:
list(data)

['train_title',
 'root',
 'alist_upload_folder',
 'amp',
 'epochs',
 'batch_n',
 'batch_size',
 'learning_rate',
 'abs_learning_rate',
 'no_better_stop',
 'checkpointing_steps',
 'label_smoothing',
 'weight_decay',
 'classify',
 'y_n',
 'debug',
 'test',
 'seed',
 'label_train',
 'label_val',
 'model',
 'grad_norm_best',
 'grad_norm',
 'total_grad_norm_best',
 'total_grad_norm',
 'train_loss_best',
 'train_loss',
 'train_acc_best',
 'train_acc',
 'train_f1_best',
 'train_f1',
 'train_recall_best',
 'train_recall',
 'train_class_acc_0_best',
 'train_class_acc_0',
 'train_class_mcc_0_best',
 'train_class_mcc_0',
 'train_class_f1_0_best',
 'train_class_f1_0',
 'train_class_recall_0_best',
 'train_class_recall_0',
 'train_class_acc_1_best',
 'train_class_acc_1',
 'train_class_mcc_1_best',
 'train_class_mcc_1',
 'train_class_f1_1_best',
 'train_class_f1_1',
 'train_class_recall_1_best',
 'train_class_recall_1',
 'val_loss_best',
 'val_loss',
 'val_acc_best',
 'val_acc',
 'val_f1_best',
 'va

In [51]:
data[['train_title', 'label_train']]

,train_title,label_train
0,20250831_2_P100_seed0_wd0.0001_IDX2,1(364471)@1(364471)
1,20250831_2_P100_seed0_wd0.001_IDX1,1(364471)@1(364471)
2,20250831_2_P100_seed0_wd0.01_IDX0,1(364471)@1(364471)
3,20250831_2_P100_seed1_wd0.0001_IDX5,1(364471)@1(364471)
4,20250831_2_P100_seed1_wd0.001_IDX4,1(364471)@1(364471)
5,20250831_2_P100_seed1_wd0.01_IDX3,1(364471)@1(364471)
6,20250831_2_P100_seed2_wd0.0001_IDX8,1(364471)@1(364471)
7,20250831_2_P100_seed2_wd0.001_IDX7,1(364471)@1(364471)
8,20250831_2_P100_seed2_wd0.01_IDX6,1(364471)@1(364471)
9,20250831_2_P100_seed3_wd0.0001_IDX11,1(364471)@1(364471)


In [52]:
# 多seed平均
key_col_name = ['train_f1']
key_col_name = ['train_loss', 'train_f1', 'val_f1', 'val_f1_best', 'val_loss', 'test_best_f1', 'label_train']
key_col_name = ['train_loss', 'train_f1', 'val_f1', 'val_f1_best', 'val_loss', 'label_train']

d = data.loc[:, key_col_name + ['cost']]
d['train_title'] = data['train_title'].apply(lambda x: '_'.join([i for i in x.split('_') if 'seed' not in i]))
d['label_train'] = d['label_train'].apply(lambda x:int(x.split('@')[-1][2:-1])*2)

# 只考察 final
_match_bool = data['train_title'].apply(lambda x: 'final' in x or ('final' not in x and 'best' not in x))
d = d.loc[_match_bool]
# 去除 final/best 的标记
d['train_title'] = d['train_title'].apply(lambda x: x.replace('_final', '').replace('_best', ''))

# # 关键字过滤 白名单
# keys = ['bc']
# _match_key_bool = data['train_title'].apply(lambda x: any([k in x for k in keys]))
# d = d.loc[_match_key_bool]
# 关键字过滤 黑名单
keys = ['wd0.0001', 'wd0.001']
_match_key_bool = data['train_title'].apply(lambda x: all([k not in x for k in keys]))
d = d.loc[_match_key_bool]

# # 根据 idx 区分类别
# # 用于未区分title的情况
# _cls = d['train_title'].apply(lambda x: [(int(i.replace('IDX', '')) % 3) for i in x.split('_') if 'IDX' in i][0])
# _cls = _cls.apply(lambda x: ['nomove30', 'nomove50', 'nomove70'][x])
# d['train_title'] = d['train_title'].apply(lambda x: '_'.join([i for i in x.split('_') if 'IDX' not in i])) + '_' + _cls

# title 已经区分
d['train_title'] = d['train_title'].apply(lambda x: '_'.join([i for i in x.split('_') if 'IDX' not in i]))

d['cost'] = d['cost'].apply(lambda x: float(x.replace('h', '')))
d = d.groupby('train_title').mean()
d['cost'] = d['cost'].apply(lambda x: str(round(x,2)) + 'h')
d.sort_values('val_f1_best',ascending=False)


,train_loss,train_f1,val_f1,val_f1_best,val_loss,label_train,cost
train_title,,,,,,,
20250901_P100_wu10,0.331192,0.996546,0.718025,0.757777,0.782993,728942.0,1.37h
20250831_2_P100_wd0.0001,0.338218,0.992322,0.713452,0.745058,0.790910,728942.0,1.22h
20250831_2_P100_wd0.01,0.338387,0.992218,0.715234,0.743055,0.786556,728942.0,1.37h
20250901_P100_wu0,0.402354,0.934481,0.696984,0.742979,0.724567,728942.0,1.36h
20250831_2_P100_wd0.001,0.338855,0.991929,0.715561,0.716554,0.786893,728942.0,1.29h


In [53]:
# 多IDX平均
key_col_name = 'val_f1'
key_col_name = 'label_train'

d = data.loc[:, [key_col_name, 'cost']]
d['train_title'] = data['train_title'].apply(lambda x: '_'.join([i for i in x.split('_') if 'IDX' not in i and 'seed' not in i ]))

d['cost'] = d['cost'].apply(lambda x: float(x.replace('h', '')))
d = d.groupby('train_title').mean()
d['cost'] = d['cost'].apply(lambda x: str(x) + 'h')
d.sort_values([key_col_name],ascending=False)

TypeError: agg function failed [how->mean,dtype->object]